In [1]:
import subprocess
import numpy as np
import wave
import scipy.signal
import scipy.ndimage
import os
import shutil
from tqdm import tqdm
import pandas as pd
from tqdm import tqdm

In [2]:
root_path = "{workspace}/my_project/src/data_origin/video"
folders = os.listdir(root_path)
folders

['training80_01',
 'training80_02',
 'training80_03',
 'training80_04',
 'training80_05',
 'training80_06',
 'training80_07',
 'training80_08',
 'training80_09',
 'training80_10',
 'training80_11',
 'training80_12']

### mp4 to wav

In [3]:
folder_path = "./wav_ffmpeg"
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
os.makedirs(folder_path, exist_ok=True)

for folder in folders:
    video_filenames = os.listdir(os.path.join(root_path, folder))
    print(folder)
    for video_filename in tqdm(video_filenames):
        mp4_file = os.path.join(root_path, folder, video_filename).replace("\\", "/")
        wav_file = os.path.join("./wav_ffmpeg", video_filename.replace(".mp4", ".wav"))
        command = [
            "ffmpeg", "-i", mp4_file,  # 입력 파일
            "-ac", "1",  # 오디오 채널 (1 = Mono)
            "-ar", "16000",  # 샘플링 레이트 16kHz
            "-q:a", "0",  # 최고 품질
            "-map", "a",  # 오디오만 추출
            wav_file
        ]
        subprocess.run(command, check=True)

training80_01


100%|██████████| 80/80 [00:04<00:00, 18.48it/s]


training80_02


100%|██████████| 80/80 [00:04<00:00, 19.44it/s]


training80_03


...

100%|██████████| 80/80 [00:04<00:00, 19.44it/s]


training80_11


100%|██████████| 80/80 [00:04<00:00, 19.49it/s]


training80_12


100%|██████████| 80/80 [00:04<00:00, 19.58it/s]


### wav to audio feature

In [4]:
wav_files = os.listdir("./wav_ffmpeg")
folder_path = "./pyaudiofeatures_30"
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
os.makedirs(folder_path, exist_ok=True)

for wav_file in tqdm(wav_files):
    # Open the WAV file
    wf = wave.open("./wav_ffmpeg/" + wav_file, 'rb')
    rate = wf.getframerate()  # Sampling rate (Hz)
    channels = wf.getnchannels()  # Number of channels
    total_frames = wf.getnframes()  # Total number of frames

    # print(f"Sample Rate: {rate}, Channels: {channels}, Frames: {total_frames}")

    # Read audio data
    audio_data = wf.readframes(total_frames)
    wf.close()

    # Convert audio data to a NumPy array
    audio_array = np.frombuffer(audio_data, dtype=np.int16)
    

    # Convert stereo to mono if necessary (average the channels)
    if channels > 1:
        audio_array = audio_array.reshape(-1, channels).mean(axis=1)

    # Normalize the audio signal
    audio_array = audio_array / np.max(np.abs(audio_array))

    # Apply Short-Time Fourier Transform (STFT)
    f, t, Zxx = scipy.signal.stft(audio_array, fs=rate, nperseg=256)
    feature = np.abs(Zxx)

    # Resize using interpolation instead of simple resizing
    feature_shape=(30, 64)
    zoom_factors = (feature_shape[0] / feature.shape[0], feature_shape[1] / feature.shape[1])
    feature_resized = scipy.ndimage.zoom(feature, zoom_factors, order=1)  # Bilinear interpolation

    # Save the extracted features as a CSV file
    save_path = os.path.join(folder_path, wav_file[:-4] + ".mp4" + ".wav_st.csv")
    df = pd.DataFrame(feature_resized)  # Convert NumPy array to DataFrame
    df.to_csv(save_path, index=False, header=False)  # Save to CSV



100%|██████████| 960/960 [00:11<00:00, 81.34it/s]
